In [ ]:
system"cd ",getenv[`HOME],"/course-introductory-workshop"
.trn.nbdir:system"cd"
\l scripts/loaddata.q

# Loading Data & IPC

**Learning objectives**

To understand:

* Saving and loading CSV files
* Saving and loading JSON files
* Connecting to a kdb+/q process
* Publishing and subscribing to a kdb+/q process

## Saving to CSV

Sometimes, we may want to export data from kdb+ into csv files. We can achieve this using the `save` function, specifying a *csv* file extension.

As we are currently inside the database directory, we will save the file one level up in the directory structure.

In [ ]:
summary:0!select num:count i, sum distance by vendor from trips where month = max month 
save `$":",.trn.nbdir,"/data/summary.csv"

You can now see this File in the Jupyter Tree, download it and open it in excel

## Loading from CSV

We may also want to import data into kdb+ from csv files.

To read in a text file, we need to specify the type of each column, along with the delimiter used to separate columns.

We can check the types in *summary* using meta.

In [ ]:
meta summary

We can use this information to load in the csv file to a table as follows

In [ ]:
newsummary:("SJF";enlist csv) 0: `$":",.trn.nbdir,"/data/summary.csv"
newsummary

In [ ]:
meta newsummary

## Saving to JSON

Data can also be exported from kdb+ into json files.

Again, we achieve this using the `save` function. However, we now use a *json* file extension.

In [ ]:
save `$":",.trn.nbdir,"/data/summary.json"

## Loading from JSON

We may also want to import data into kdb+ from json files. We achieve this using the `load` function. 

Note that this will replace current variable *summary* with the loaded version.

In [ ]:
load `$":",.trn.nbdir,"/data/summary.json"
summary

Unlike CSV, JSON has some idea of data-types. However, since everything in JSON is either an *number* or a *string*, the `vendor` and `num` columns have changed type.

In [ ]:
meta summary

We can convert them to *long* and *symbol* respectively, using `update`.

In [ ]:
update "S"$vendor, "j"$num from `summary
meta summary

# QIPC

A kdb+ database is often part of a larger environment, connecting to other processes to send/receive data and queries/results.

Communication between processes is managed via tcp/ip.
-	a server process listens for connections on a port, and processes any requests 
-	a client process initiates a connection to a server and sends commands to be executed

Any q process can act as a client, a server, or both.

Communications can be synchronous (wait for a result to be returned) or asynchronous (no wait and no result returned). 

## Initializing a q server 

A q server is initialized simply by specifying the port to listen on.

In [ ]:
\p 5000

One way to access this kdb+/q server is via a web browser.

The environment (variables and tables) can be inspected using
* `http://localhost:5000`

We can also execute queries directly from the url, for example
- `http://localhost:5000/?count trips`
- `http://localhost:5000/?select avg tip by vendor from trips`

## Running a kdb+/q client 

We can also connect to a kdb+/q server from a q client.

First we will start a server process, listening on port 5001.

In [ ]:
system"nohup q ",.trn.nbdir,"/scripts/server.q -p 5001 >/dev/null 2>&1 &"

Then we will connect using `hopen`.

In [ ]:
h:hopen 5001
h

The result is a *handle*, which we will use to communicate with the server.

The simplest way to communicate is to send q commands, as strings, to execute on the server.

For example, to get a list of tables.

In [ ]:
h"tables[]"

We can then inspect the *populations* table

In [ ]:
h"populations"

and execute queries

In [ ]:
h"select sum population from populations" 

If we do not require a result, we can send an `async` message using the negative handle.

For example, to set a variable on the server (with an *async* message)

In [ ]:
neg[h]"a:123"

Which we can then inspect (with a sync message)

In [ ]:
h"a"

## Publishing/Subscribing

Async messaging is often used to publish messages to subscriber processes.

For an example, we will subscribe to the server process to receive *apprequests* messages (i.e. requests for pickup).

In [ ]:
h"sub[]"

The server will now begin sending us *apprequests* messages, which we can process in the (currently undefined) function `upd`.

First we will get the schema from the server and define a local table to store updates

In [ ]:
show apprequests:h"apprequests"
meta apprequests

This table is initially empty

In [ ]:
apprequests

But we will define the callback function, `upd`, to handle the incoming updates.

In [ ]:
upd:{`apprequests upsert x;}

We can then see the `apprequests` table begin to grow

In [ ]:
apprequests

We can query the table directly to get the total number of requests per vendor

In [ ]:
select num:count i by vendor from apprequests

However, more efficient to aggregate the data on arrival to keep a real-time count.

We will first define a table to store results, and then update the `upd` function.

In [ ]:
numrequests:select num:count i by vendor from apprequests

upd:{
  `apprequests upsert x;
  numrequests[x`vendor]+:1
  }

In [ ]:
numrequests

Using the `upd` function above, we can create an alert. Let's say we want to publish an alert if the vendor is for BBB and the longitude is greater than -74.0 so we can inform the driver of their next trip:

In [ ]:
upd:{
    .debug.x:x;
  `apprequests upsert x;
  numrequests[x`vendor]+:1;
    if[`BBB = x`vendor;
        if[-74.0<x`long;
            0N!"A trip for BBB has arrived at the long of ",string x`long
        ]
    ];
  }

This table should now update in real-time as updates arrive. We can also close the connection via the command `hclose`. This will prevent any further updates to the local table, `apprequests` & `numrequests`.

In [ ]:
hclose h

# KX Introductory Workshop Certification 

Thank you for attending KX Introductory Workshop in which we hope you learned a lot of new concepts about kdb+/q. 

You can now close the sandbox and go back to the KX Academy. Remember to leave feedback on the [course homepage](https://learninghub.kx.com/courses/kdb-developer-level-1/) and complete the short quiz to receive the **KX Introductory Workshop Certification**. To receive the certificate, you will need to get 70% in the quiz. 